In [1]:
'''

This code is part of the SIPN2 project focused on improving sub-seasonal to seasonal predictions of Arctic Sea Ice. 
If you use this code for a publication or presentation, please cite the reference in the README.md on the
main page (https://github.com/NicWayand/ESIO). 

Questions or comments should be addressed to nicway@uw.edu

Copyright (c) 2018 Nic Wayand

GNU General Public License v3.0


'''

# Standard Imports
%matplotlib inline
%load_ext autoreload
%autoreload
import matplotlib
import scipy
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import numpy as np
import pandas as pd
import xarray as xr
import xesmf as xe
import os
import glob
import seaborn as sns
import warnings
import datetime
warnings.simplefilter(action='ignore', category=FutureWarning)

# ESIO Imports

from esio import EsioData as ed
from esio import import_data
from esio import ice_plot

In [2]:
# General plotting settings
sns.set_style('whitegrid')
sns.set_context("talk", font_scale=1.5, rc={"lines.linewidth": 2.5})

In [14]:
E = ed.EsioData.load()
# Directories
all_models=['nicosipn']
runType='forecast'
updateall = False

In [4]:
stero_grid_file = E.obs['NSIDC_0051']['grid']
obs_grid = import_data.load_grid_info(stero_grid_file, model='NSIDC')
# Ensure latitude is within bounds (-90 to 90)
# Have to do this because grid file has 90.000001
obs_grid['lat_b'] = obs_grid.lat_b.where(obs_grid.lat_b < 90, other = 90)

In [5]:
obs_grid

<xarray.Dataset>
Dimensions:  (ni: 448, ni_b: 449, nj: 304, nj_b: 305)
Coordinates:
  * nj       (nj) int64 0 1 2 3 4 5 6 7 8 ... 296 297 298 299 300 301 302 303
  * ni       (ni) int64 0 1 2 3 4 5 6 7 8 ... 440 441 442 443 444 445 446 447
  * nj_b     (nj_b) int64 0 1 2 3 4 5 6 7 8 ... 297 298 299 300 301 302 303 304
  * ni_b     (ni_b) int64 0 1 2 3 4 5 6 7 8 ... 441 442 443 444 445 446 447 448
Data variables:
    lat      (nj, ni) float64 31.1 31.25 31.4 31.55 ... 34.92 34.77 34.62 34.47
    lon      (nj, ni) float64 168.3 168.4 168.5 168.7 ... -9.745 -9.872 -9.999
    lat_b    (nj_b, ni_b) float64 31.13 31.28 31.42 31.57 ... 34.49 34.35 31.37
    lon_b    (nj_b, ni_b) float64 168.5 168.6 168.7 ... -9.846 -9.972 102.3
    imask    (nj, ni) bool True True True True True ... True True True True True

In [6]:
def load_sun_extent(f, valid_time):
    #extent maps in uint8
    with open( f, 'rb') as frr:
        extentforecast = np.fromfile(frr, dtype=np.uint8)
    extentforecast = extentforecast.reshape(448, 304)
    da = xr.DataArray(extentforecast, dims=('ni','nj')).T
    oceanmask = da<255 # Mask land
    da = (da==250).astype('int').where(oceanmask)
    da.coords['valid_time'] = valid_time
    
    return da
    

In [7]:
def load_sun_sic(f, valid_time):
    #extent maps in uint8
    with open( f, 'rb') as frr:
        extentforecast = np.fromfile(frr, dtype=np.uint8)
    extentforecast = extentforecast.reshape(448, 304)
    da = xr.DataArray(extentforecast, dims=('ni','nj')).T
    oceanmask = da<255 # Mask land
    da = (da/250).where(oceanmask)
    da.coords['valid_time'] = valid_time
    
    return da
    

In [8]:
cvar = 'SIC'

# Pre 2018 July
prefix_single = ['SIPN2_Extent_over']
# Post 2018 July
ens_prefix = ['SIPN2_SIC_High','SIPN2_SIC_Low','SIPN2_SIC_Mean']
#  2019 June ANOTHER CHANGE!!! sigh
# 001 is low, 002 is mean and 003 is high 
ens_prefix = ['001','002','003'] 

In [62]:
for model in all_models:
    print('Regridding ', model, '...')
    
    data_dir = os.path.join('/home/disk/sipn/upload/',model,runType) #,'*','netcdf') 
    data_out = '/home/disk/sipn/nicway/data/model/nicosipn/forecast/sipn_nc'
   
    # Get list of dir ("months")
    mon_dirs = [ name for name in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, name)) ]
    print('mon_dirs are ',mon_dirs)

    print("Found ",len(mon_dirs)," months.")
    if updateall:
        print("Updating all files...")
    else:
        print("Only updating new files")
    
    # for each  month
    for cdir in mon_dirs:
        print("processing ",cdir)
        cyear = cdir.split('_')[0]
        cmon = cdir.split('_')[1]
        print("   year, months is ",cyear, cmon)

        # Check if already imported and skip (unless updateall flag is True)
        f_out = os.path.join(data_out, cyear+'_'+cmon+'_Stereo.nc') # netcdf file out 
        print('Considering whether to make: ',f_out)
        if not updateall:
            # TODO: Test if the file is openable (not corrupted)
            if os.path.isfile(f_out):
                print("Skipping ", os.path.basename(f_out), " already imported.")
                continue # Skip, file already imported
        
        #cyear = '2018'
        #cmon = '08'
        #cdir = '2018_08'
        #ens_prefix = ['SIPN2_SIC_High','SIPN2_SIC_Low','SIPN2_SIC_Mean']
        
        if (cyear=='2018') & (cmon=='07'): # Single ensemble
            prefix_l = prefix_single
            Nunderscores = 4
        else:
            prefix_l = ens_prefix
            Nunderscores = 3 # Becuase file format changes.... sigh

        


        ds_ens = []
        for (en, c_prefix) in enumerate(prefix_l):
                
            if (cyear>='2019'):
                en_file = os.path.join(data_dir, cdir, 'netcdf','NicoSun_2019_00'+str(en+1)+'_concentration.nc')
                print("File Name: ",en_file)
                ds = xr.open_mfdataset(en_file, autoclose=True)
                da=ds.siconc
                da=da/100
                DS = np.datetime64(datetime.datetime(int(cyear), int(cmon), 1, 0, 0))
                DE = DS + np.timedelta64(len(da.time), 'D')
                #valid_dates = pd.date_range(start=DS,periods=len(da.time),freq='D') # does not work, not sure why
                valid_dates = np.arange ( DS, DE , dtype='datetime64[D]')
                da.coords['valid_time'] = xr.DataArray(valid_dates,dims='time')
                
                da = da.swap_dims({'time':'valid_time'})
                da = da.rename({'xaxis':'ni'})
                da = da.rename({'yaxis':'nj'})
                da = da.drop('time')
                #print(da)
                ds_out = da

            else:    # prior to 2019 each valid time is in a separte binary file
                all_files = sorted(glob.glob(os.path.join(data_dir, cdir, cvar, c_prefix+'*.bin'), recursive=True))
                print(os.path.join(data_dir, cdir, cvar, c_prefix+'*.bin'))
                print("Found",len(all_files),"files")

                ds_l = []
                for cf in all_files:
                    valid_date = os.path.basename(cf).split('_')[Nunderscores].split('.')[0]
                    print(valid_date)
                    valid_date = np.datetime64(pd.to_datetime(valid_date))
                    print(valid_date)
                    if (cyear=='2018') & (cmon=='07'): # Single ensemble
                        da_f = load_sun_extent(cf, valid_date)
                    else:
                        da_f = load_sun_sic(cf, valid_date)

                    ds_l.append(da_f)
                    
                ds_out = xr.concat(ds_l, dim='valid_time')
                print(ds_out)
                                
            # Set ensemeble dim
            ds_out.coords['ensemble'] = en
            
            # add init time
            #print('ds_out before adding stuff: \n\n\n\n',ds_out)
            ds_out.coords['init_time'] = ds_out.valid_time.values[0]
            ds_out.coords['fore_time'] = (ds_out.valid_time - ds_out.init_time)
            ds_out = ds_out.swap_dims({'valid_time':'fore_time'})
            ds_out = ds_out.drop('valid_time')

            ds_out.name = 'sic'

            # Expand dims
            ds_out = import_data.expand_to_sipn_dims(ds_out)
            #print('ds_out after expanding to sipn: \n\n\n\n',ds_out)

            # Add lat/lon
            ds_out.coords['lat'] = obs_grid.lat
            ds_out.coords['lon'] = obs_grid.lon
            
            
            # Store in list
            ds_ens.append(ds_out)
            
            if len(ds_ens)>1:
                ds_out_all = xr.concat(ds_ens, dim='ensemble')
            else:
                ds_out_all = ds_ens[0]

        # # Save regridded to netcdf file
        ds_out_all.to_netcdf(f_out)
        ds_out_all = None # Memory clean up
        ds = None
        print('Saved ', f_out)
        

Regridding  nicosipn ...
mon_dirs are  ['2018_08', '2019_06', '2018_07']
Found  3  months.
Only updating new files
processing  2018_08
   year, months is  2018 08
Considering whether to make:  /home/disk/sipn/nicway/data/model/nicosipn/forecast/sipn_nc/2018_08_Stereo.nc
Skipping  2018_08_Stereo.nc  already imported.
processing  2019_06
   year, months is  2019 06
Considering whether to make:  /home/disk/sipn/nicway/data/model/nicosipn/forecast/sipn_nc/2019_06_Stereo.nc
File Name:  /home/disk/sipn/upload/nicosipn/forecast/2019_06/netcdf/NicoSun_2019_001_concentration.nc
<xarray.DataArray 'siconc' (valid_time: 122, ni: 448, nj: 304)>
dask.array<shape=(122, 448, 304), dtype=float64, chunksize=(122, 448, 304)>
Coordinates:
  * valid_time  (valid_time) datetime64[ns] 2019-06-01 2019-06-02 ... 2019-09-30
Dimensions without coordinates: ni, nj
File Name:  /home/disk/sipn/upload/nicosipn/forecast/2019_06/netcdf/NicoSun_2019_002_concentration.nc
<xarray.DataArray 'siconc' (valid_time: 122, ni: 

In [ ]:
ds_out

# Plotting

In [ ]:
sic_all = xr.open_mfdataset('/home/disk/sipn/nicway/data/model/nicosipn/forecast/sipn_nc/2018_08_Stereo.nc')
sic_all

In [ ]:

# Set up plotting info
cmap_sic = matplotlib.colors.ListedColormap(sns.color_palette("Blues", 10))
cmap_sic.set_bad(color = 'red')

# # Plot original projection
# plt.figure(figsize=(20,10))
# ax1 = plt.axes(projection=ccrs.PlateCarree())
# ds_p = ds.sic.isel(fore_time=8).isel(ensemble=0)
# ds_p.plot.pcolormesh(ax=ax1, x='lon', y='lat', 
#                                  vmin=0, vmax=1,
#                                  cmap=matplotlib.colors.ListedColormap(sns.color_palette("Blues", 10)),
#                     transform=ccrs.PlateCarree());
# ax1.set_extent([-180, 180, -90, 90], crs=ccrs.PlateCarree())
# gl = ax1.gridlines(crs=ccrs.PlateCarree(), linestyle='-')
# gl.xlabels_bottom = True
# gl.ylabels_left = True
# gl.xformatter = LONGITUDE_FORMATTER
# gl.yformatter = LATITUDE_FORMATTER
# ax1.coastlines(linewidth=0.75, color='black', resolution='50m');

# # Plot SIC on target projection
# (f, ax1) = ice_plot.polar_axis()
# ds_p.plot.pcolormesh(ax=ax1, x='lon', y='lat', 
#                                      transform=ccrs.PlateCarree(),
#                                      cmap=cmap_sic)
# ax1.set_title('Original Grid')

# Plot SIC on target projection
(f, ax1) = ice_plot.polar_axis()
ds_p2 = sic_all.sic.isel(init_time=0).isel(fore_time=8).isel(ensemble=0)
ds_p2.plot.pcolormesh(ax=ax1, x='lon', y='lat', 
                                     transform=ccrs.PlateCarree(),
                                     cmap=cmap_sic)
ax1.set_title('Target Grid')

(f, ax1) = ice_plot.polar_axis()
ds_p2 = sic_all.sic.isel(init_time=0).isel(fore_time=8).isel(ensemble=1)
ds_p2.plot.pcolormesh(ax=ax1, x='lon', y='lat', 
                                     transform=ccrs.PlateCarree(),
                                     cmap=cmap_sic)
ax1.set_title('Target Grid')

(f, ax1) = ice_plot.polar_axis()
ds_p2 = sic_all.sic.isel(init_time=0).isel(fore_time=8).isel(ensemble=2)
ds_p2.plot.pcolormesh(ax=ax1, x='lon', y='lat', 
                                     transform=ccrs.PlateCarree(),
                                     cmap=cmap_sic)
ax1.set_title('Target Grid')
